## 1. Configuración e Imports

In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

# Agregar src al path
project_dir = Path.cwd().parent

# Agregar src al path explícitamente para imports internos
src_dir = project_dir / 'src'
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))
if str(project_dir) not in sys.path:
    sys.path.append(str(project_dir))

from src.data_loading import load_processed_data, save_object
from src.features import create_tfidf_features, get_top_tfidf_words
from scipy.sparse import hstack, csr_matrix
import os

print('✓ Librerías y módulos cargados')

✓ Librerías y módulos cargados


## 2. Cargar Datos Procesados

In [2]:
# Cargar datos procesados usando src
df_train = load_processed_data('train_processed.csv')
df_test = load_processed_data('test_processed.csv')

print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")
print(f"\nColumnas: {list(df_train.columns)}")

✓ Datos procesados cargados: 1596781 filas
✓ Datos procesados cargados: 359 filas
Train: (1596781, 11)
Test: (359, 11)

Columnas: ['polarity', 'text', 'text_clean', 'length', 'num_words', 'num_hashtags', 'num_mentions', 'num_urls', 'num_uppercase', 'pct_uppercase', 'num_intensified']


## 3. Configurar Stopwords Personalizadas

**Decisión del EDA:** Mantener palabras que aportan sentimiento como "not", "no", "very"

In [3]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Palabras que SÍ aportan sentimiento (las sacamos de stopwords)
sentiment_words = {
    'not', 'no', 'nor', 'neither', 'never', 'none', 'nobody', 'nothing', 'nowhere',
    'very', 'really', 'so', 'too', 'quite', 'rather',
    'but', 'however', 'although', 'though',
    'all', 'every', 'any', 'some', 'most',
    'much', 'many', 'more', 'most', 'less', 'least',
    'good', 'bad', 'best', 'worst', 'better', 'worse'
}

# Stopwords finales = stopwords default - palabras de sentimiento
custom_stopwords = ENGLISH_STOP_WORDS - sentiment_words

print(f"Stopwords originales: {len(ENGLISH_STOP_WORDS)}")
print(f"Stopwords personalizadas: {len(custom_stopwords)}")
print(f"Palabras de sentimiento conservadas: {len(sentiment_words)}")
print(f"\nEjemplos conservados: {list(sentiment_words)[:10]}")

Stopwords originales: 318
Stopwords personalizadas: 291
Palabras de sentimiento conservadas: 35

Ejemplos conservados: ['neither', 'best', 'really', 'some', 'every', 'nowhere', 'no', 'nor', 'though', 'however']


## 4. Crear Vectorizador TF-IDF con Bigramas

**Configuración:**
- **ngram_range=(1,2)**: palabras individuales + bigramas
- **max_features=5000**: limitar vocabulario
- **min_df=5**: palabra debe aparecer al menos 5 veces
- **max_df=0.8**: palabra no puede estar en más del 80% de tweets

In [4]:
# La configuración del vectorizador se pasa directamente a la función create_tfidf_features de src

## 5. Entrenar TF-IDF en TRAIN

In [5]:
# Crear features TF-IDF usando src
# Esta función entrena en train y transforma test automáticamente
X_train_tfidf, X_test_tfidf, tfidf_vectorizer = create_tfidf_features(
    train_texts=df_train['text_clean'].fillna(''),
    test_texts=df_test['text_clean'].fillna(''),
    max_features=10000,
    ngram_range=(1, 2),
    stop_words=list(custom_stopwords),
    min_df=5,
    max_df=0.8
)

print(f"\nShape Train: {X_train_tfidf.shape}")
print(f"Shape Test: {X_test_tfidf.shape}")


Shape Train: (1596781, 10000)
Shape Test: (359, 10000)


## 6. Transformar TEST

**Nota**: La transformación de TEST ya se realizó automáticamente en la celda anterior 
mediante `create_tfidf_features()`, que entrena en TRAIN y transforma TEST sin data leakage.

## 7. Inspeccionar Vocabulario Aprendido

Verificamos que incluya bigramas importantes como "not good", "very happy"

In [6]:
# Obtener términos y sus índices
feature_names = tfidf_vectorizer.get_feature_names_out()

# Separar unigramas y bigramas
unigramas = [term for term in feature_names if ' ' not in term]
bigramas = [term for term in feature_names if ' ' in term]

print(f"=== VOCABULARIO ===")
print(f"\nTotal términos: {len(feature_names)}")
print(f"  - Unigramas: {len(unigramas)}")
print(f"  - Bigramas: {len(bigramas)}")

print(f"\n📝 Ejemplos de bigramas importantes:")
sentiment_bigrams = [b for b in bigramas if any(word in b for word in ['not', 'no', 'very', 'really', 'so', 'too'])]
print(sentiment_bigrams[:20])

=== VOCABULARIO ===

Total términos: 10000
  - Unigramas: 6121
  - Bigramas: 3879

📝 Ejemplos de bigramas importantes:
['absolutely love', 'absolutely nothing', 'actually really', 'all annoying', 'all know', 'all not', 'all really', 'all so', 'all songs', 'all too', 'all very', 'amazing so', 'amp no', 'amp not', 'amp so', 'amp some', 'annoying help', 'anytime soon', 'ate some', 'ate too']


In [7]:
# Verificar bigramas críticos de sentimiento
critical_bigrams = ['not good', 'not bad', 'very happy', 'very sad', 
                    'dont like', 'cant wait', 'so sad', 'so happy']

print("\n🔍 Verificando bigramas críticos:")
for bigram in critical_bigrams:
    if bigram in tfidf_vectorizer.vocabulary_:
        print(f"  ✓ '{bigram}' está en el vocabulario")
    else:
        print(f"  ✗ '{bigram}' NO está en el vocabulario")


🔍 Verificando bigramas críticos:
  ✓ 'not good' está en el vocabulario
  ✓ 'not bad' está en el vocabulario
  ✓ 'very happy' está en el vocabulario
  ✓ 'very sad' está en el vocabulario
  ✓ 'dont like' está en el vocabulario
  ✗ 'cant wait' NO está en el vocabulario
  ✓ 'so sad' está en el vocabulario
  ✓ 'so happy' está en el vocabulario


## 7.1 Top Features por Clase usando TF-IDF

Analizamos qué términos son más relevantes para cada clase (Positivo vs Negativo).

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Calcular promedios de TF-IDF por clase
positive_tweets = df_train[df_train['polarity'] == 4]['text_clean'].fillna('')
negative_tweets = df_train[df_train['polarity'] == 0]['text_clean'].fillna('')

# Entrenar vectorizadores separados para análisis exploratorio
print("Analizando top features por clase...")
vec_pos = TfidfVectorizer(max_features=20, ngram_range=(1,2), stop_words=list(custom_stopwords))
vec_neg = TfidfVectorizer(max_features=20, ngram_range=(1,2), stop_words=list(custom_stopwords))

X_pos = vec_pos.fit_transform(positive_tweets)
X_neg = vec_neg.fit_transform(negative_tweets)

print("🟢 TOP 10 TÉRMINOS EN TWEETS POSITIVOS:")
for term in list(vec_pos.vocabulary_.keys())[:10]:
    print(f"  • {term}")

print("\n🔴 TOP 10 TÉRMINOS EN TWEETS NEGATIVOS:")
for term in list(vec_neg.vocabulary_.keys())[:10]:
    print(f"  • {term}")

Analizando top features por clase...
🟢 TOP 10 TÉRMINOS EN TWEETS POSITIVOS:
  • love
  • im
  • thanks
  • got
  • too
  • just
  • so
  • time
  • like
  • day

🔴 TOP 10 TÉRMINOS EN TWEETS NEGATIVOS:
  • got
  • day
  • today
  • like
  • no
  • not
  • all
  • im
  • just
  • but


In [9]:
# Top términos usando src
top_words_df = get_top_tfidf_words(tfidf_vectorizer, X_train_tfidf, n_top=20)

print("\n🔝 Top 20 términos más importantes (TF-IDF promedio):")
print(top_words_df)


🔝 Top 20 términos más importantes (TF-IDF promedio):
      word  tfidf_score
0       im     0.018156
1       so     0.015834
2     just     0.014049
3      but     0.013185
4      not     0.012851
5     good     0.011934
6      day     0.011151
7       no     0.010250
8      all     0.010094
9     like     0.010013
10    love     0.009807
11    work     0.009619
12     too     0.009448
13   going     0.009181
14    dont     0.009137
15   today     0.009093
16     got     0.008427
17     lol     0.008413
18  thanks     0.008405
19    time     0.008002


## 8. Preparar Features Numéricas

Las 7 features extraídas en el preprocesamiento

In [10]:
# Columnas de features numéricas
numeric_features = ['length', 'num_words', 'num_hashtags', 'num_mentions', 
                   'num_urls', 'num_uppercase', 'pct_uppercase']

# Extraer features numéricas
X_train_numeric = df_train[numeric_features].values
X_test_numeric = df_test[numeric_features].values

print(f"Features numéricas train: {X_train_numeric.shape}")
print(f"Features numéricas test: {X_test_numeric.shape}")

print(f"\n📊 Estadísticas features numéricas (TRAIN):")
print(df_train[numeric_features].describe().round(2))

Features numéricas train: (1596781, 7)
Features numéricas test: (359, 7)

📊 Estadísticas features numéricas (TRAIN):
           length   num_words  num_hashtags  num_mentions    num_urls  \
count  1596781.00  1596781.00    1596781.00    1596781.00  1596781.00   
mean        74.20       13.20          0.03          0.49        0.05   
std         36.38        6.94          0.22          0.59        0.23   
min          6.00        1.00          0.00          0.00        0.00   
25%         44.00        7.00          0.00          0.00        0.00   
50%         69.00       12.00          0.00          0.00        0.00   
75%        104.00       19.00          0.00          1.00        0.00   
max        374.00       64.00         24.00         12.00        5.00   

       num_uppercase  pct_uppercase  
count     1596781.00     1596781.00  
mean            3.26           6.22  
std             5.21           9.77  
min             0.00           0.00  
25%             1.00           1.16

## 9. Combinar TF-IDF + Features Numéricas

Matriz final = [TF-IDF texto] + [7 features numéricas]

In [11]:
print("Combinando features...\n")

# Convertir features numéricas a sparse matrix
X_train_numeric_sparse = csr_matrix(X_train_numeric)
X_test_numeric_sparse = csr_matrix(X_test_numeric)

# Concatenar horizontalmente (TF-IDF + numeric)
X_train_final = hstack([X_train_tfidf, X_train_numeric_sparse])
X_test_final = hstack([X_test_tfidf, X_test_numeric_sparse])

print("✓ Features combinadas")
print(f"\nTRAIN:")
print(f"  - TF-IDF: {X_train_tfidf.shape}")
print(f"  - Numéricas: {X_train_numeric.shape}")
print(f"  - FINAL: {X_train_final.shape}")

print(f"\nTEST:")
print(f"  - TF-IDF: {X_test_tfidf.shape}")
print(f"  - Numéricas: {X_test_numeric.shape}")
print(f"  - FINAL: {X_test_final.shape}")

Combinando features...

✓ Features combinadas

TRAIN:
  - TF-IDF: (1596781, 10000)
  - Numéricas: (1596781, 7)
  - FINAL: (1596781, 10007)

TEST:
  - TF-IDF: (359, 10000)
  - Numéricas: (359, 7)
  - FINAL: (359, 10007)


In [12]:
import sys

print(f"\n💾 USO DE MEMORIA:")
print(f"  X_train_final: {X_train_final.data.nbytes / 1024**2:.1f} MB")
print(f"  X_test_final: {X_test_final.data.nbytes / 1024**2:.1f} MB")
print(f"  Equivalente denso: {(X_train_final.shape[0] * X_train_final.shape[1] * 8) / 1024**3:.1f} GB")


💾 USO DE MEMORIA:
  X_train_final: 132.8 MB
  X_test_final: 0.0 MB
  Equivalente denso: 119.1 GB


## 10. Preparar Etiquetas (y)

In [13]:
# Convertir polaridad 0/4 a 0/1 para clasificación binaria
y_train = (df_train['polarity'].values == 4).astype(int)
y_test = (df_test['polarity'].values == 4).astype(int)

print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

print(f"\nBalance TRAIN:")
print(f"  - Negativos (0): {(y_train == 0).sum()}")
print(f"  - Positivos (1): {(y_train == 1).sum()}")

print(f"\nBalance TEST:")
print(f"  - Negativos (0): {(y_test == 0).sum()}")
print(f"  - Positivos (1): {(y_test == 1).sum()}")

y_train: (1596781,)
y_test: (359,)

Balance TRAIN:
  - Negativos (0): 798383
  - Positivos (1): 798398

Balance TEST:
  - Negativos (0): 177
  - Positivos (1): 182


## 11. Guardar Vectores y Objetos

In [14]:
# Guardar objetos usando src
import os
os.makedirs('../data/vectorized', exist_ok=True)

save_object(X_train_final, '../vectorized/X_train.pkl')
save_object(X_test_final, '../vectorized/X_test.pkl')
save_object(y_train, '../vectorized/y_train.pkl')
save_object(y_test, '../vectorized/y_test.pkl')
save_object(tfidf_vectorizer, '../vectorized/tfidf_vectorizer.pkl')

print("✓ Datos vectorizados guardados")

✓ Objeto guardado en: d:\Diplomatura en ia\trabajo practico 3 -Omar Gonzalez\tp3_nlp_sentiment\data\processed\..\vectorized\X_train.pkl
✓ Objeto guardado en: d:\Diplomatura en ia\trabajo practico 3 -Omar Gonzalez\tp3_nlp_sentiment\data\processed\..\vectorized\X_test.pkl
✓ Objeto guardado en: d:\Diplomatura en ia\trabajo practico 3 -Omar Gonzalez\tp3_nlp_sentiment\data\processed\..\vectorized\y_train.pkl
✓ Objeto guardado en: d:\Diplomatura en ia\trabajo practico 3 -Omar Gonzalez\tp3_nlp_sentiment\data\processed\..\vectorized\y_test.pkl
✓ Objeto guardado en: d:\Diplomatura en ia\trabajo practico 3 -Omar Gonzalez\tp3_nlp_sentiment\data\processed\..\vectorized\tfidf_vectorizer.pkl
✓ Datos vectorizados guardados


In [15]:
## 9.1 Cobertura de Vocabulario en TEST

# Términos únicos en test
test_vocab = set()
for text in df_test['text_clean'].fillna(''):
    test_vocab.update(text.split())

# Términos del vectorizador presentes en test
vocab_terms = set(tfidf_vectorizer.vocabulary_.keys())
overlap = test_vocab.intersection(vocab_terms)

print(f"\n📊 COBERTURA DE VOCABULARIO:")
print(f"  Términos únicos en TEST: {len(test_vocab)}")
print(f"  Términos en vocabulario TF-IDF: {len(vocab_terms)}")
print(f"  Términos en común: {len(overlap)}")
print(f"  Cobertura: {len(overlap)/len(test_vocab)*100:.1f}%")


📊 COBERTURA DE VOCABULARIO:
  Términos únicos en TEST: 1584
  Términos en vocabulario TF-IDF: 10000
  Términos en común: 988
  Cobertura: 62.4%


## 12. Resumen Final

In [16]:
print("="*60)
print("RESUMEN DE VECTORIZACIÓN")
print("="*60)

print(f"\n📊 DATOS FINALES:")
print(f"  Train: {X_train_final.shape[0]:,} tweets × {X_train_final.shape[1]:,} features")
print(f"  Test:  {X_test_final.shape[0]:,} tweets × {X_test_final.shape[1]:,} features")

print(f"\n🔤 COMPOSICIÓN DE FEATURES:")
print(f"  - Vocabulario TF-IDF: {len(tfidf_vectorizer.vocabulary_):,} términos")
print(f"    * Unigramas: {len(unigramas):,}")
print(f"    * Bigramas: {len(bigramas):,}")
print(f"  - Features numéricas: 7")
print(f"  - TOTAL: {X_train_final.shape[1]:,} features")

print(f"\n✅ DECISIONES DEL EDA IMPLEMENTADAS:")
print(f"  ✓ TF-IDF con bigramas ('not good', 'very happy')")
print(f"  ✓ Stopwords personalizadas (conservamos 'not', 'no', 'very')")
print(f"  ✓ 7 features numéricas integradas")
print(f"  ✓ Vocabulario limitado a {X_train_tfidf.shape[1]:,} features")
print(f"  ✓ Matriz sparse eficiente (sparsity: {(1 - X_train_final.nnz / (X_train_final.shape[0] * X_train_final.shape[1])) * 100:.2f}%)")

print(f"\n🎯 LISTO PARA MODELADO")
print(f"  Próximo paso: Notebook 4 - Entrenar modelos")

RESUMEN DE VECTORIZACIÓN

📊 DATOS FINALES:
  Train: 1,596,781 tweets × 10,007 features
  Test:  359 tweets × 10,007 features

🔤 COMPOSICIÓN DE FEATURES:
  - Vocabulario TF-IDF: 10,000 términos
    * Unigramas: 6,121
    * Bigramas: 3,879
  - Features numéricas: 7
  - TOTAL: 10,007 features

✅ DECISIONES DEL EDA IMPLEMENTADAS:
  ✓ TF-IDF con bigramas ('not good', 'very happy')
  ✓ Stopwords personalizadas (conservamos 'not', 'no', 'very')
  ✓ 7 features numéricas integradas
  ✓ Vocabulario limitado a 10,000 features
  ✓ Matriz sparse eficiente (sparsity: 99.89%)

🎯 LISTO PARA MODELADO
  Próximo paso: Notebook 4 - Entrenar modelos


## 13. Word2Vec Embeddings

Word2Vec es otra técnica de vectorización que genera embeddings densos (vectores de 100-300 dimensiones) donde palabras similares quedan cerca en el espacio vectorial.

A diferencia de TF-IDF que usa frecuencias, Word2Vec aprende representaciones semánticas del contexto en que aparecen las palabras.

In [ ]:
# =============================================================
# ENTRENAMIENTO DEL MODELO WORD2VEC (ejecutar solo si es necesario)
# =============================================================
# El modelo ya está entrenado y guardado en models/word2vec_model.pkl
# Descomentar solo si necesitás re-entrenar.

# from gensim.models import Word2Vec
# import pandas as pd
#
# # Cargar datos preprocesados
# df_train = pd.read_csv('../data/processed/train_processed.csv')
# sentences = [text.split() for text in df_train['text_clean'].fillna('').tolist()]
#
# # Entrenar Word2Vec
# model_w2v = Word2Vec(
#     sentences=sentences,
#     vector_size=100,
#     window=5,
#     min_count=5,
#     workers=4,
#     epochs=10
# )
#
# # Guardar modelo
# model_w2v.save('../models/word2vec_model.pkl')
# print(f"✓ Modelo guardado con {len(model_w2v.wv):,} palabras")

In [1]:
# Verificar modelo guardado
from gensim.models import Word2Vec

model_w2v = Word2Vec.load('../models/word2vec_model.pkl')
print(f"✅ Modelo Word2Vec cargado: {len(model_w2v.wv):,} palabras")
print(f"   Vector size: {model_w2v.wv.vector_size} dimensiones")

# Ejemplo de palabras similares
print(f"\n📊 Ejemplo - Palabras similares a 'happy':")
for word, score in model_w2v.wv.most_similar('happy', topn=5):
    print(f"   {word}: {score:.4f}")

✅ Modelo Word2Vec cargado: 57,795 palabras
   Vector size: 100 dimensiones

📊 Ejemplo - Palabras similares a 'happy':
   happyy: 0.6584
   thrilled: 0.6207
   pleased: 0.6005
   sad: 0.5942
   upset: 0.5836
